## Table of Contents

* [Data Import and Data Cleaning](#data-import-and-data-cleaning)
    * [Import Libraries](#import-libraries)
    * [Import CSV Files](#import-csv-files)
    * [Check Data](#check-data)
    * [Clean Up Data](#clean-up-data)
    * [Functions Created to Fill NaNs and Map Variables to Numeric Values](#functions-created-to-fill-nans-and-map-variables-to-numeric-values)
    * [Manual Mapping](#manual-mapping)
    * [Find Correlations and Drop Columns](#find-correlations-and-drop-columns)
* [Save Clean Data as CSV Files](#save-clean-data-as-csv-files)
* [Load Clean CSV File](#load-clean-csv-file)
* [Dummify Columns for Train and Test](#dummify-columns-for-train-and-test)
    * [Get Dummies on All Nominal Columns](#get-dummies-on-all-nominal-columns)
* [Save Data With Dummies as CSV File](#save-data-with-dummies-as-csv-file)

# Data Import and Data Cleaning

## Import Libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import statsmodels.api as sm

## Import CSV Files

In [6]:
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

## Check Data

In [7]:
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


## Clean Up Data

In [8]:
# Change train columns to lowercase and snakecase
train.columns = train.columns.str.lower().str.replace(' ', '_')

# Change test columns to lowercase and snakecase
test.columns = test.columns.str.lower().str.replace(' ', '_')

In [9]:
# Check columns that have more than 0 NaN for both train.csv and test.csv

# Train
null_counts = train.isnull().sum()
columns_with_missing_values = null_counts[null_counts > 0]
print(columns_with_missing_values)

# Test
null_counts = test.isnull().sum()
columns_with_missing_values = null_counts[null_counts > 0]
print(columns_with_missing_values)

lot_frontage       330
alley             1911
mas_vnr_type        22
mas_vnr_area        22
bsmt_qual           55
bsmt_cond           55
bsmt_exposure       58
bsmtfin_type_1      55
bsmtfin_sf_1         1
bsmtfin_type_2      56
bsmtfin_sf_2         1
bsmt_unf_sf          1
total_bsmt_sf        1
bsmt_full_bath       2
bsmt_half_bath       2
fireplace_qu      1000
garage_type        113
garage_yr_blt      114
garage_finish      114
garage_cars          1
garage_area          1
garage_qual        114
garage_cond        114
pool_qc           2042
fence             1651
misc_feature      1986
dtype: int64
lot_frontage      160
alley             820
mas_vnr_type        1
mas_vnr_area        1
bsmt_qual          25
bsmt_cond          25
bsmt_exposure      25
bsmtfin_type_1     25
bsmtfin_type_2     25
electrical          1
fireplace_qu      422
garage_type        44
garage_yr_blt      45
garage_finish      45
garage_qual        45
garage_cond        45
pool_qc           874
fence          

## Functions Created to fill NaNs and Map Variables to Numeric Values

In [10]:
# Create function to change nominal columns with NaN/empty to 'NA' to
# match data dictionary

# Train function
def train_change_nominal(column):
    train[column] = train[column].replace(np.nan, 'NA')
    return train

# Test function
def test_change_nominal(column):
    test[column] = test[column].replace(np.nan, 'NA')
    return test

In [11]:
# Use function for training data
train_change_nominal('alley')
train_change_nominal('mas_vnr_type')
train_change_nominal('bsmt_qual')
train_change_nominal('bsmt_cond')
train_change_nominal('bsmt_exposure')
train_change_nominal('bsmtfin_type_1')
train_change_nominal('bsmtfin_type_2')
train_change_nominal('fireplace_qu')
train_change_nominal('garage_type')
train_change_nominal('garage_finish')
train_change_nominal('garage_qual')
train_change_nominal('garage_cond')
train_change_nominal('pool_qc')
train_change_nominal('fence')
train_change_nominal('misc_feature')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,NA,IR1,HLS,...,0,0,NA,NA,NA,0,1,2008,WD,298751
2047,785,905377130,30,RL,NaN,12342,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.0,7558,Pave,NA,Reg,Bnk,...,0,0,NA,NA,NA,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.0,10400,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,11,2009,WD,144000


In [12]:
# Use function for testing data
test_change_nominal('alley')
test_change_nominal('mas_vnr_type')
test_change_nominal('bsmt_qual')
test_change_nominal('bsmt_cond')
test_change_nominal('bsmt_exposure')
test_change_nominal('bsmtfin_type_1')
test_change_nominal('bsmtfin_type_2')
test_change_nominal('fireplace_qu')
test_change_nominal('garage_type')
test_change_nominal('garage_finish')
test_change_nominal('garage_qual')
test_change_nominal('garage_cond')
test_change_nominal('pool_qc')
test_change_nominal('fence')
test_change_nominal('misc_feature')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NA,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NA,IR1,Lvl,...,0,185,0,NA,NA,NA,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NA,Reg,Lvl,...,0,0,0,NA,MnPrv,NA,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NA,Reg,Lvl,...,0,0,0,NA,GdWo,NA,0,5,2007,WD


In [13]:
# Created function to change continuous columns with NaN/empty to 0

# Train function
def train_change_continuous(column):
    train[column] = train[column].fillna(0)
    return train

# Test function
def test_change_continuous(column):
    test[column] = test[column].fillna(0)
    return test

In [14]:
# Use function for training data
train_change_continuous('lot_frontage')
train_change_continuous('mas_vnr_area')
train_change_continuous('bsmtfin_sf_1')
train_change_continuous('bsmtfin_sf_2')
train_change_continuous('bsmt_unf_sf')
train_change_continuous('total_bsmt_sf')
train_change_continuous('bsmt_full_bath')
train_change_continuous('bsmt_half_bath')
train_change_continuous('garage_area')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,0.0,13517,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,NA,IR1,HLS,...,0,0,NA,NA,NA,0,1,2008,WD,298751
2047,785,905377130,30,RL,0.0,12342,Pave,NA,IR1,Lvl,...,0,0,NA,NA,NA,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.0,7558,Pave,NA,Reg,Bnk,...,0,0,NA,NA,NA,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.0,10400,Pave,NA,Reg,Lvl,...,0,0,NA,NA,NA,0,11,2009,WD,144000


In [15]:
# Use function for testing data
test_change_continuous('lot_frontage')
test_change_continuous('mas_vnr_area')
test_change_continuous('bsmtfin_sf_1')
test_change_continuous('bsmtfin_sf_2')
test_change_continuous('bsmt_unf_sf')
test_change_continuous('total_bsmt_sf')
test_change_continuous('bsmt_full_bath')
test_change_continuous('bsmt_half_bath')
test_change_continuous('garage_area')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NA,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,NA,IR1,Lvl,...,0,185,0,NA,NA,NA,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NA,Reg,Lvl,...,0,0,0,NA,MnPrv,NA,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,NA,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NA,Reg,Lvl,...,0,0,0,NA,GdWo,NA,0,5,2007,WD


In [16]:
# Created function to map ordinal columns with numeric values

# Train function
def train_convert_column(column):
    train[column] = train[column].replace(np.nan, 'NA')
    mapping = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    train[column] = train[column].map(mapping)
    return train

# Test function
def test_convert_column(column):
    test[column] = test[column].replace(np.nan, 'NA')
    mapping = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
    test[column] = test[column].map(mapping)
    return test

In [17]:
# Use function for training data
train_convert_column('exter_qual')
train_convert_column('exter_cond')
train_convert_column('bsmt_qual')
train_convert_column('bsmt_cond')
train_convert_column('heating_qc')
train_convert_column('kitchen_qual')
train_convert_column('garage_qual')
train_convert_column('garage_cond')
train_convert_column('pool_qc')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,0.0,13517,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,0,3,2010,WD,138500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1587,921126030,20,RL,79.0,11449,Pave,NA,IR1,HLS,...,0,0,0,NA,NA,0,1,2008,WD,298751
2047,785,905377130,30,RL,0.0,12342,Pave,NA,IR1,Lvl,...,0,0,0,NA,NA,0,3,2009,WD,82500
2048,916,909253010,50,RL,57.0,7558,Pave,NA,Reg,Bnk,...,0,0,0,NA,NA,0,3,2009,WD,177000
2049,639,535179160,20,RL,80.0,10400,Pave,NA,Reg,Lvl,...,0,0,0,NA,NA,0,11,2009,WD,144000


In [18]:
# Use function for testing data
test_convert_column('exter_qual')
test_convert_column('exter_cond')
test_convert_column('bsmt_qual')
test_convert_column('bsmt_cond')
test_convert_column('heating_qc')
test_convert_column('kitchen_qual')
test_convert_column('garage_qual')
test_convert_column('garage_cond')
test_convert_column('pool_qc')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,0,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,NA,IR1,Lvl,...,0,0,0,0,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,...,0,0,0,0,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,NA,IR1,Lvl,...,0,185,0,0,NA,NA,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NA,Reg,Lvl,...,0,0,0,0,MnPrv,NA,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NA,Reg,Lvl,...,0,0,0,0,GdWo,NA,0,5,2007,WD


In [19]:
# Create function to map bsmtfin

# Train function
def train_map_type(column):
    mapping = {'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
    train[column] = train[column].map(mapping)
    return train

# Test function
def test_map_type(column):
    mapping = {'NA': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
    test[column] = test[column].map(mapping)
    return test

In [20]:
# Use function on training data
train_map_type('bsmtfin_type_1')
train_map_type('bsmtfin_type_2')

# Use function on testing data
test_map_type('bsmtfin_type_1')
test_map_type('bsmtfin_type_2')

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,0,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,NA,IR1,Lvl,...,0,0,0,0,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,...,0,0,0,0,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,NA,IR1,Lvl,...,0,185,0,0,NA,NA,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NA,Reg,Lvl,...,0,0,0,0,MnPrv,NA,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NA,Reg,Lvl,...,0,0,0,0,NA,NA,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NA,Reg,Lvl,...,0,0,0,0,GdWo,NA,0,5,2007,WD


## Manual Mapping

In [21]:
# Map garage_finish to numerical values
# Define mapping
garage_finish_mapping = {'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

# Map for train data
train['garage_finish'] = train['garage_finish'].map(garage_finish_mapping)

# Map for test data
test['garage_finish'] = test['garage_finish'].map(garage_finish_mapping)

In [22]:
# Map lot_shape to numerical values
# Define mapping
lot_shape_mapping = {'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3}

# Map for train data
train['lot_shape'] = train['lot_shape'].map(lot_shape_mapping)

# Map for test data
test['lot_shape'] = test['lot_shape'].map(lot_shape_mapping)

In [23]:
# Map utilities to numerical values
# Define mapping
utilities_mapping = {'ELO': 0, 'NoSeWa': 1, 'NoSewr': 2, 'AllPub': 3}

# Map for train data
train['utilities'] = train['utilities'].map(utilities_mapping)

# Map for test data
test['utilities'] = test['utilities'].map(utilities_mapping)

In [24]:
# Map paved_drive to numerical values
# Define mapping
paved_drive_mapping = {'N': 0, 'P': 1, 'Y': 2}

# Map for train data
train['paved_drive'] = train['paved_drive'].map(paved_drive_mapping)

# Map for test data
test['paved_drive'] = test['paved_drive'].map(paved_drive_mapping)

In [25]:
# Map functional to numerical values
# Define mapping
functional_mapping = {'Sal': 0, 'Sev': 1, 'Maj2': 2, 'Maj1': 3, 'Mod': 4, 'Min2': 5, 'Min1': 6, 'Typ': 7}

# Map for train data
train['functional'] = train['functional'].map(functional_mapping)

# Map for test data
test['functional'] = test['functional'].map(functional_mapping)

In [26]:
train['ms_zoning'].unique()
# Map ms_zoning to data dictionary values
# Define mapping
ms_zoning_mapping = {'RM': 'RM', 'RP': 'RP', 'RL': 'RL', 'RH': 'RH', 'I (all)': 'I', 'FV': 'FV', 'C (all)': 'C', 'A (agr)': 'A'}

# Map for train data
train['ms_zoning'] = train['ms_zoning'].map(ms_zoning_mapping)

# Map for test data
test['ms_zoning'] = test['ms_zoning'].map(ms_zoning_mapping)

In [27]:
# Map electrical to numerical values and fix test NaN
# Define mapping
electrical_mapping = {'Mix': 0, 'FuseP': 1, 'FuseF': 2, 'FuseA': 3, 'SBrkr': 4}

# Map for train data
train['electrical'] = train['electrical'].map(electrical_mapping)

# Replace NaN
test.loc[:, 'electrical'] = test['electrical'].replace(np.nan, 'Mix')

# Map for test data
test['electrical'] = test['electrical'].map(electrical_mapping)

## Find Correlations and Drop Columns

In [28]:
train.corr()

C:\Users\Crypt\AppData\Local\Temp\ipykernel_24504\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,id,pid,ms_subclass,lot_frontage,lot_area,lot_shape,utilities,overall_qual,overall_cond,year_built,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,pool_qc,misc_val,mo_sold,yr_sold,saleprice
id,1.000000,0.175793,0.026543,-0.013538,0.032872,-0.011705,0.018313,-0.061483,-0.026096,-0.064444,...,0.001382,0.033747,-0.022791,0.009758,0.055696,0.050720,-0.012683,0.127723,-0.975747,-0.051398
pid,0.175793,1.000000,-0.003632,-0.038402,0.024135,0.093960,-0.031622,-0.265863,0.106861,-0.347039,...,-0.081129,0.150179,-0.024679,-0.042030,0.005825,0.021364,0.004223,-0.032735,0.008476,-0.255052
ms_subclass,0.026543,-0.003632,1.000000,-0.216259,-0.245484,0.075306,0.023312,0.035763,-0.070141,0.035983,...,-0.020289,-0.039842,-0.030088,-0.038819,-0.004585,-0.009025,-0.027485,0.013027,-0.032870,-0.087335
lot_frontage,-0.013538,-0.038402,-0.216259,1.000000,0.135586,0.174018,0.019333,0.114469,-0.032452,0.020571,...,0.095295,0.009207,0.008076,0.036598,0.092288,0.102806,0.026066,-0.006382,0.007713,0.181456
lot_area,0.032872,0.024135,-0.245484,0.135586,1.000000,-0.301763,-0.029802,0.105824,-0.019185,0.036002,...,0.140864,0.014139,0.019553,0.067714,0.115102,0.129200,0.093922,0.003197,-0.029454,0.296566
lot_shape,-0.011705,0.093960,0.075306,0.174018,-0.301763,1.000000,0.013735,-0.249357,0.077207,-0.277834,...,-0.084296,0.090935,-0.028078,-0.069171,-0.039590,-0.052713,-0.049819,-0.005869,0.034830,-0.294542
utilities,0.018313,-0.031622,0.023312,0.019333,-0.029802,0.013735,1.000000,0.030044,0.006142,0.029184,...,-0.017370,0.011181,0.003045,-0.071728,0.001881,0.001896,0.002666,0.049178,-0.027663,0.026404
overall_qual,-0.061483,-0.265863,0.035763,0.114469,0.105824,-0.249357,0.030044,1.000000,-0.082770,0.602964,...,0.308855,-0.154554,0.031938,0.048752,0.006558,0.019568,0.022099,0.019242,-0.011578,0.800207
overall_cond,-0.026096,0.106861,-0.070141,-0.032452,-0.019185,0.077207,0.006142,-0.082770,1.000000,-0.370988,...,-0.052266,0.108320,0.026907,0.047359,-0.005806,-0.008272,0.014269,-0.003144,0.047664,-0.097019
year_built,-0.064444,-0.347039,0.035983,0.020571,0.036002,-0.277834,0.029184,0.602964,-0.370988,1.000000,...,0.207798,-0.380082,0.016104,-0.037866,0.003728,0.008823,0.000626,-0.007083,-0.003559,0.571849


In [29]:
# Find correlations to reduce multicollinearity
garage_corr = train[['garage_area', 'garage_cars']].corr()
print(garage_corr)
area_corr = train[['total_bsmt_sf', '1st_flr_sf', 'gr_liv_area']].corr()
print(area_corr)

             garage_area  garage_cars
garage_area      1.00000      0.89318
garage_cars      0.89318      1.00000
               total_bsmt_sf  1st_flr_sf  gr_liv_area
total_bsmt_sf       1.000000    0.808351     0.454245
1st_flr_sf          0.808351    1.000000     0.562441
gr_liv_area         0.454245    0.562441     1.000000


In [30]:
# Dropping garage_cars to reduce multicollinarity

# Drop for train data
train.drop(columns=['garage_cars'], inplace=True)

# Drop for test data
test.drop(columns=['garage_cars'], inplace=True)

In [31]:
# According to the data dictionary, year_remod/add is same as construction
# date if no remodeling so dropping garage_yr_blt to avoid redudancy

# Drop for train data
train.drop(columns=['garage_yr_blt'], inplace=True)

# Drop for test data
test.drop(columns=['garage_yr_blt'], inplace=True)

In [32]:
# Dropping PID because when submitting/creating model, we will be using ID to represent the house

# Drop for train data
train.drop(columns=['pid'], inplace=True)

# Drop for test data
test.drop(columns=['pid'], inplace=True)

In [33]:
train.head()

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,60,RL,0.0,13517,Pave,NA,2,Lvl,3,...,0,0,0,NA,NA,0,3,2010,WD,130500
1,544,60,RL,43.0,11492,Pave,NA,2,Lvl,3,...,0,0,0,NA,NA,0,4,2009,WD,220000
2,153,20,RL,68.0,7922,Pave,NA,3,Lvl,3,...,0,0,0,NA,NA,0,1,2010,WD,109000
3,318,60,RL,73.0,9802,Pave,NA,3,Lvl,3,...,0,0,0,NA,NA,0,4,2010,WD,174000
4,255,50,RL,82.0,14235,Pave,NA,2,Lvl,3,...,0,0,0,NA,NA,0,3,2010,WD,138500


# Save Clean Data As CSV Files

In [34]:
# Save Train Data
train.to_csv('../clean_datasets/finish_clean_training.csv', index=False)

# Save Test Data
test.to_csv('../clean_datasets/finish_clean_testing.csv', index=False)

# Load Clean CSV File

In [35]:
# Load Clean Training Data
train_cleaned = pd.read_csv('../clean_datasets/finish_clean_training.csv')

# Load Clean Testing Data
test_cleaned = pd.read_csv('../clean_datasets/finish_clean_testing.csv')

# Dummify Columns for Train and Test

## Get Dummies On All Nominal Columns

In [36]:
# Train Get Dummies
train_cleaned = pd.get_dummies(columns=['ms_subclass',
                                        'ms_zoning',
                                        'street',
                                        'alley',
                                        'land_contour',
                                        'lot_config',
                                        'neighborhood',
                                        'condition_1',
                                        'condition_2',
                                        'bldg_type',
                                        'house_style',
                                        'roof_style',
                                        'roof_matl',
                                        'exterior_1st',
                                        'exterior_2nd',
                                        'mas_vnr_type',
                                        'foundation',
                                        'heating',
                                        'central_air',
                                        'garage_type',
                                        'misc_feature',
                                        'sale_type'], drop_first=True, data=train_cleaned)

In [37]:
# Test Get Dummies
test_cleaned = pd.get_dummies(columns=['ms_subclass',
                                        'ms_zoning',
                                        'street',
                                        'alley',
                                        'land_contour',
                                        'lot_config',
                                        'neighborhood',
                                        'condition_1',
                                        'condition_2',
                                        'bldg_type',
                                        'house_style',
                                        'roof_style',
                                        'roof_matl',
                                        'exterior_1st',
                                        'exterior_2nd',
                                        'mas_vnr_type',
                                        'foundation',
                                        'heating',
                                        'central_air',
                                        'garage_type',
                                        'misc_feature',
                                        'sale_type'], drop_first=True, data=test_cleaned)

# Save Data With Dummies as CSV File

In [38]:
# Save Train Data
train_cleaned.to_csv('../clean_datasets/finish_clean_training_dummified.csv', index=False)

# Save Test Data
test_cleaned.to_csv('../clean_datasets/finish_clean_testing_dummified.csv', index=False)